In [1]:
# Importing needed libraries
import matplotlib.pyplot as plt
import numpy as np
import h5py
import cv2
import io
import easyocr

In [2]:
from keras.models import load_model

from timeit import default_timer as timer

"""
Start of:
Setting up full paths

"""

In [3]:
# Full or absolute path to 'Section4' with preprocessed datasets
# (!) On Windows, the path should look like following:
# r'C:\Users\your_name\PycharmProjects\CNNCourse\Section4'
# or:
# 'C:\\Users\\your_name\\PycharmProjects\\CNNCourse\\Section4'

full_path_to_Section4 = \
    'C:/Users/yelti/THESIS-FINAL/CNN/Section4'

In [ ]:
# Full or absolute path to 'Section5' with designed models
# (!) On Windows, the path should look like following:
# r'C:\Users\your_name\PycharmProjects\CNNCourse\Section5'
# or:
# 'C:\\Users\\your_name\\PycharmProjects\\CNNCourse\\Section5'

full_path_to_Section5 = \
    'C:/Users/yelti/THESIS-FINAL/CNN/Section4/Section5'

"""
Start of:
Loading saved model

"""

In [ ]:
# Loading model
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +

model = load_model(full_path_to_Section5 + '/' +
                   'custom' + '/' +
                   'License_Model.h5')
                   # Check point
print("Model loaded")

"""
Start of:
Loading and assigning best weights

"""

In [ ]:
# Loading and assigning best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +

model.load_weights(full_path_to_Section5 + '/' + 'custom' + '/' + 'License_Model.h5')

# Check point
print('Best weights are loaded and assigned')

"""
Start of:
Preparing labels

"""

In [ ]:
# Defining list with labels for custom dataset
labels = ['LicensePlateN','LicensePlateO']

# Check point
print(labels)

"""
Start of:
Loading saved Mean Image and Standard Deviation

"""

In [ ]:
# Opening saved Mean Image for GRAY custom dataset
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section5 + '/' +
                'custom' + '/' +
                'mean_rgb_dataset_custom.hdf5', 'r') as f:
    # Extracting saved array for Mean Image
    # Saving it into new variable
    mean_gray = f['mean']  # HDF5 dataset
    # Converting it into Numpy array
    mean_gray = np.array(mean_gray)  # Numpy arrays

In [ ]:
# Opening saved Standard Deviation for GRAY custom dataset
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' +
               'custom' + '/' +
               'std_rgb_dataset_custom.hdf5', 'r') as f:
    # Extracting saved array for Standard Deviation
    # Saving it into new variable
    std_gray = f['std']  # HDF5 dataset
    # Converting it into Numpy array
    std_gray = np.array(std_gray)  # Numpy arrays

"""
Start of:
Preparing function to plot bar chart

"""

In [ ]:
# Defining function to plot bar chart with scores values
def bar_chart(obtained_scores, classes_names):
     # Arranging X axis
     x_positions = np.arange(obtained_scores.size)

     # Creating bar chart
     bars = plt.bar(x_positions, obtained_scores, align='center', alpha=0.6)

     # Highlighting the highest bar
     bars[np.argmax(obtained_scores)].set_color('red')

     # Giving labels to bars along X axis
     plt.xticks(x_positions, classes_names, rotation=10, fontsize=15)

     # Giving names to axes
     plt.xlabel('Class', fontsize=20)
     plt.ylabel('Value', fontsize=20)

     # Giving name to bar chart
     plt.title('Obtained Scores', fontsize=20)

     # Adjusting borders of the plot
     plt.tight_layout(pad=2.5)

     # Initializing object of the buffer
     b = io.BytesIO()

     # Saving bar chart into the buffer
     plt.savefig(b, format='png', dpi=200)

     # Closing plot with bar chart
     plt.close()

     # Moving pointer to the beginning of the buffer
     b.seek(0)

     # Reading bar chart from the buffer
     bar_image = np.frombuffer(b.getvalue(), dtype=np.uint8)

     # Closing buffer
     b.close()

     # Decoding buffer
     bar_image = cv2.imdecode(bar_image, 1)

     # Returning Numpy array with bar chart
     return bar_image


# Check point
print('Function to plot Bar Chart is successfully defined')

"""
Start of:
Preparing OpenCV windows to be shown

"""

In [ ]:
# Giving names to the windows
# Specifying that windows are resizable

# Window to show current view from camera in Real Time
cv2.namedWindow('Current view', cv2.WINDOW_NORMAL)

# Window to show cut fragment
cv2.namedWindow('Cut fragment', cv2.WINDOW_NORMAL)

#  # Window to show classification result
# cv2.namedWindow('Classified as', cv2.WINDOW_NORMAL)

#  # Window to show bar chart with scores
# cv2.namedWindow('Scores', cv2.WINDOW_NORMAL)

# Check point
print('OpenCV windows are ready')

"""
Start of:
Reading frames from camera in the loop

"""

In [ ]:
# Defining 'VideoCapture' object
# to read stream video from camera
# Index of the built-in camera is usually 0
# Try to select other cameras by passing 1, 2, 3, etc.
camera = cv2.VideoCapture(0)


# Defining counter for FPS (Frames Per Second)
counter = 0

# Starting timer for FPS
# Getting current time point in seconds
fps_start = timer()


# Creating image with black background
temp = np.zeros((720, 1280, 3), np.uint8)


# Defining loop to catch frames
while True:
    # Capturing frames one-by-one from camera
    _, frame_bgr = camera.read()

    """
    Start of:
    Detecting object
    """

    # Converting caught frame to HSV colour space
    frame_hsv = cv2.cvtColor(frame_bgr, cv2.IMREAD_GRAYSCALE)

    # Applying mask with founded boundary numbers
    # gray = cv2.cvtColor(frame_hsv, cv2.IMREAD_GRAYSCALE)
    # bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(frame_hsv, 30, 200)
    #mask = np.zeros(frame_hsv, np.uint8)

    # Finding contours
    # All found contours are placed into a list
    # Every individual contour is a Numpy array of (x, y) coordinates,
    # that represent boundary points of detected object
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # contours, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # Sorting contours from biggest to smallest
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    """
    End of:
    Detecting object
    """

    """
    Start of:
    Classifying detected object
    """

    # If any contour is found, extracting coordinates of the biggest one
    if contours:
        # Getting rectangle coordinates and spatial size of the biggest contour
        # Function 'cv2.boundingRect()' returns an approximate rectangle,
        # that covers the region around found contour
        (x_min, y_min, box_width, box_height) = cv2.boundingRect(contours[0])

        # Drawing obtained rectangle on the current BGR frame
        cv2.rectangle(frame_bgr, (x_min, y_min),
                      (x_min + box_width, y_min + box_height),
                      (230, 161, 0), 3)

        # # Putting text above rectangle
        # cv2.putText(frame_bgr, 'DETECTED', (x_min - 5, y_min - 25),
        #             cv2.FONT_HERSHEY_SIMPLEX, 1.0, (230, 161, 0), 2)

        """
        Start of:
        Cutting detected fragment
        """

        # Cutting detected fragment from BGR frame
        cut_fragment_bgr = frame_bgr[y_min + int(box_height * 0.1):
                                     y_min + box_height - int(box_height * 0.1),
                                     x_min + int(box_width * 0.1):
                                     x_min + box_width - int(box_width * 0.1)]

        """
        End of:
        Cutting detected fragment
        """

        """
        Start of:
        Preprocessing cut fragment
        """

        # Converting frame to GRAY by OpenCV function
        frame_gray = cv2.cvtColor(cut_fragment_bgr, cv2.COLOR_BGR2GRAY)

        # Resizing frame to 64 by 64 pixels size
        frame_gray = cv2.resize(frame_gray,
                                (100, 100),
                                interpolation=cv2.INTER_CUBIC)

        # # Extending dimension from (height, width) to (height, width, 1)
        frame_gray = frame_gray[:, :, np.newaxis]

        # # Implementing normalization by dividing image's pixels on 255.0
        frame_gray_255 = frame_gray / 255.0

        # # Implementing normalization by subtracting Mean Image
        frame_gray_255_mean = frame_gray_255 - mean_gray

        # # Implementing preprocessing by dividing on Standard Deviation
        frame_gray_255_mean_std = frame_gray_255_mean / std_gray

        # # Extending dimension from (height, width, 1)
        # # to (1, height, width, 1)
        frame_gray_255_mean_std = frame_gray_255_mean_std[np.newaxis, :, :, :]

        
        # reader = easyocr.Reader(['en'])
        # result = reader.readtext(frame_gray)
        # text = result[0][-2]
        # # print(text)



        """
        End of:
        Preprocessing cut fragment
        """

        """
        Start of:
        Implementing forward pass
        """

        # # Testing RGB custom model trained on dataset:
        # # dataset_custom_gray_255_mean_std.hdf5
        # # Caught frame is preprocessed in the same way
        # # Measuring classification time
        start = timer()
        scores = model.predict(frame_gray_255_mean_std)
        end = timer()

        # # Scores are given as 5 numbers of predictions for each class
        # # Getting index of only one class with maximum value
        prediction = np.argmax(scores)

        """
        End of:
        Implementing forward pass
        """

        """
        Start of:
        Showing OpenCV windows
        """

        # Showing current view from camera in Real Time
        # Pay attention! 'cv2.imshow' takes images in BGR format
        cv2.imshow('Current view', frame_bgr)

        # Showing cut fragment
        cv2.imshow('Cut fragment', cut_fragment_bgr)

        # Changing background to BGR(230, 161, 0)
        # B = 230, G = 161, R = 0
        temp[:, :, 0] = 230
        temp[:, :, 1] = 161
        temp[:, :, 2] = 0

        # # Adding text with current label
        cv2.putText(temp, labels[int(prediction)], (100, 200),
                     cv2.FONT_HERSHEY_TRIPLEX, 6, (255, 255, 255), 6, cv2.LINE_AA)

        # # Adding text with obtained confidence score to image with label
        cv2.putText(temp, 'Score : ' + '{0:.5f}'.format(scores[0][prediction]),
                     (100, 450), cv2.FONT_HERSHEY_DUPLEX, 4, (255, 255, 255),
                     4, cv2.LINE_AA)

        # # Adding text with time spent for classification to image with label
        cv2.putText(temp, 'Time  : ' + '{0:.5f}'.format(end - start),
                     (100, 600), cv2.FONT_HERSHEY_DUPLEX, 4, (255, 255, 255),
                     4, cv2.LINE_AA)

        # # Showing image with respect to classification results
        cv2.imshow('Classified as', temp)

        # # Showing bar chart
        cv2.imshow('Scores', bar_chart(scores[0], labels))

        """
        End of:
        Showing OpenCV windows
        """

    # If no contour is found, showing OpenCV windows with information
    else:
        # Showing current view from camera in Real Time
        # Pay attention! 'cv2.imshow' takes images in BGR format
        cv2.imshow('Current view', frame_bgr)

        # Changing background to BGR(230, 161, 0)
        # B = 230, G = 161, R = 0
        temp[:, :, 0] = 230
        temp[:, :, 1] = 161
        temp[:, :, 2] = 0

        # Adding text with information
        cv2.putText(temp, 'No object', (100, 450),
                    cv2.FONT_HERSHEY_DUPLEX, 4, (255, 255, 255), 4, cv2.LINE_AA)

        # Showing information in prepared OpenCV windows
        cv2.imshow('Cut fragment', temp)
        cv2.imshow('Classified as', temp)
        cv2.imshow('Scores', temp)

    """
    End of:
    Classifying detected object
    """

    """
    Start of:
    Calculating FPS
    """

    # Increasing counter for FPS
    counter += 1

    # Stopping timer for FPS
    # Getting current time point in seconds
    fps_stop = timer()

    # Checking if timer reached 1 second
    # Comparing
    if fps_stop - fps_start >= 1.0:
        # # Showing FPS rate
        # print('FPS rate is: ', counter)

        # Reset FPS counter
        counter = 0

        # Restart timer for FPS
        # Getting current time point in seconds
        fps_start = timer()

    """
    End of:
    Calculating FPS
    """

    # Breaking the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

"""
End of:
Reading frames from camera in the loop
"""


# Releasing camera
camera.release()

# Destroying all opened OpenCV windows
cv2.destroyAllWindows()